# 🧬 Solution_RD: Genetic Algorithm Individual for Resource Distribution

This notebook documents the `Solution_RD` class, a domain-specific implementation of the `Individual` abstract base class tailored for solving a resource distribution problem involving participants, houses, and courses. The objective balances logistical and social criteria using a flexible and pluggable design.

## 🧱 Class Overview

- `Solution_RD`: A concrete subclass of `Individual` that represents a candidate solution for assigning participants to houses and courses.
  - Representation encodes both **house allocation** and **course enrollment**.
  - Fitness balances two components:
    - **Social fitness**: Encourages participant mixing across courses.
    - **Logistic fitness**: Penalizes long distances between participant houses (based on a distance matrix).
  - Customizable fitness trade-off via `alpha` and `beta`.

## ⚙️ Parameters

- `data_matrix`: A 2D numpy array of distances between participants.
- `account_for_previous_fitness`: If `True`, tracks fitness changes over time.
- `probability_social_mutation`: Probability of performing a social mutation.
- `alpha`, `beta`: Weights controlling the importance of social vs. logistic fitness.

## ✅ Features Demonstrated

1. **Random Initialization**
   - Genome includes both house and course assignments.
   - Ensures constraints like house capacity and number of houses are respected.

2. **Fitness Evaluation**
   - Calculated as a weighted sum of:
     - Social interaction score (maximize diversity in house-course assignments).
     - Logistic penalty (minimize travel distances).
   - Supports lazy evaluation and caching.

3. **Mutation**
   - Mutation type (social or logistic) chosen probabilistically.
   - Mutates in-place.

4. **Crossover**
   - Combines genome segments from two parents (to be implemented or overridden if necessary).

5. **Parameterization**
   - Adjust evolutionary pressure via `alpha`, `beta`, and mutation probabilities.

6. **Fitness History**
   - Tracks `previous_fitness` and `next_fitness` for advanced evolutionary strategies.

## 🧠 Design Highlights

- Extends `Individual` with problem-specific logic, maintaining modularity.
- Balances domain constraints with genetic flexibility.
- Supports experimentation with different fitness weights and mutation strategies.

This class is suitable for constrained optimization tasks in social or logistical planning domains and can be integrated seamlessly into the larger genetic algorithm framework.


# Data Ingestion

In [1]:
import os
import pandas as pd
os.chdir(os.pardir) # comment if you are running this more than once
data_matrix_df = pd.read_csv('data/distance_matrix_official.csv', index_col=0)
data_matrix_np = data_matrix_df.to_numpy()
data_matrix_np.shape

(68, 68)

# Testing Features

## Create two individuals


In [2]:
from Genetic_algorithm.fitness import ResourceFitness
from Genetic_algorithm.genome import Genome
from Genetic_algorithm.solution_rd import SolutionRD
from Genetic_algorithm.mutations import logistic_mutation, social_mutation, logistic_mutation_2
from Genetic_algorithm.crossovers import social_crossover, logistic_crossover_2, full_crossover

my_fitness = ResourceFitness(data_matrix_np)
my_genome = Genome()

first_solution = SolutionRD(my_fitness, Genome, [logistic_mutation, social_mutation], [logistic_crossover_2, social_crossover])
secomnd_solution = SolutionRD(my_fitness, Genome, [logistic_mutation, social_mutation], [logistic_crossover_2, social_crossover])

## Crossover


In [3]:
for _ in range(1000):
    child1, child2 = first_solution @ secomnd_solution
    child1.mutation()
print("Fitness (child):", float(child1))
print("Fitness (child):", float(child2))

Fitness (child): 3.205545487495469
Fitness (child): 2.845772594752187


## Mutation


In [4]:
my_fitness.partecipation_matrix(genome=child1.genome)

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
       [1, 1, 1, 0, 1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 1, 0, 1, 0, 0, 0],
       [1, 0, 1, 1, 1, 0, 1, 1, 0, 0],
       [1, 0, 0, 1, 1, 1, 1, 1, 0, 0]])

In [5]:
mutant = child1.copy_Individual()
mutant.mutation()
print("original Individual:", child1.genome)
print("Mutated Individual:", mutant.genome)
print("Fitness (original):", float(child1))
print("Fitness (mutated):", float(mutant))

original Individual: 0-1110221704-19528-13689760453-1-11293574061-1-182
Mutated Individual: 0-1110221704-19528-13689760453-1-11293574061-1-182
Fitness (original): 3.205545487495469
Fitness (mutated): 3.205545487495469


## Iteration (generate mutated versions) (has to be resolved)

In [6]:
print("\nMultiple Mutations:")
for i, m in enumerate(child1):
    print(f"Mutation {i+1}:", m, "Fitness:", m)


Multiple Mutations:
Mutation 1: Fitness: 3.290089470061941 Fitness: Fitness: 3.290089470061941
Mutation 2: Fitness: 2.003034682080925 Fitness: Fitness: 2.003034682080925
Mutation 3: Fitness: 2.610565754964406 Fitness: Fitness: 2.610565754964406
Mutation 4: Fitness: 2.7882051282051283 Fitness: Fitness: 2.7882051282051283
Mutation 5: Fitness: 2.868257756563246 Fitness: Fitness: 2.868257756563246


## Operator Overloading

In [7]:
print("\nUsing ** Operator for Self-Crossover")
clones = child1 ** 3
for i, c1, in enumerate(clones):
    print(f"Clone {i+1}:", c1, "Fitness:", c1)
   


Using ** Operator for Self-Crossover
Clone 1: (<Genetic_algorithm.solution_rd.SolutionRD object at 0x10d36a9c0>, <Genetic_algorithm.solution_rd.SolutionRD object at 0x10d366240>) Fitness: (<Genetic_algorithm.solution_rd.SolutionRD object at 0x10d36a9c0>, <Genetic_algorithm.solution_rd.SolutionRD object at 0x10d366240>)
Clone 2: (<Genetic_algorithm.solution_rd.SolutionRD object at 0x10d35cce0>, <Genetic_algorithm.solution_rd.SolutionRD object at 0x10d31e390>) Fitness: (<Genetic_algorithm.solution_rd.SolutionRD object at 0x10d35cce0>, <Genetic_algorithm.solution_rd.SolutionRD object at 0x10d31e390>)
Clone 3: (<Genetic_algorithm.solution_rd.SolutionRD object at 0x10d35c290>, <Genetic_algorithm.solution_rd.SolutionRD object at 0x10d35c230>) Fitness: (<Genetic_algorithm.solution_rd.SolutionRD object at 0x10d35c290>, <Genetic_algorithm.solution_rd.SolutionRD object at 0x10d35c230>)


## Comparison


In [8]:
print("\nIs Individual 1 better than Individual 2?", child1 > child2)


Is Individual 1 better than Individual 2? True


## Sorting and Deduplication via Set

In [9]:
print("\n=== Sorting Individuals by Fitness ===")
individuals = [child for child in child1]

for i, ind in enumerate(individuals):
    print(f"Ind {i+1}: Genome = {ind.genome}, Fitness = {float(ind)}")

sorted_individuals = sorted(individuals)
print("\nSorted by fitness (ascending):")
for i, ind in enumerate(sorted_individuals):
    print(f"Rank {i+1}: Genome = {ind.genome}, Fitness = {float(ind)}")


=== Sorting Individuals by Fitness ===
Ind 1: Genome = 2-1012109657-128034-1134-185-1791260826-139-140157, Fitness = 2.949334516415262
Ind 2: Genome = 10-1212065321084-1-1793409-126715-1897450281-16-13, Fitness = 3.455725769755802
Ind 3: Genome = 11-102024521-19780-1363618947-10-152-123780-145196, Fitness = 2.3432198499711485
Ind 4: Genome = -12011026370418295-1-181603-14-12579-148597-162013, Fitness = 2.730977620730271
Ind 5: Genome = 0-1220110739-184-12561-192603-151748025186-14739-1, Fitness = 2.0472140762463344

Sorted by fitness (ascending):
Rank 1: Genome = 0-1220110739-184-12561-192603-151748025186-14739-1, Fitness = 2.0472140762463344
Rank 2: Genome = 11-102024521-19780-1363618947-10-152-123780-145196, Fitness = 2.3432198499711485
Rank 3: Genome = -12011026370418295-1-181603-14-12579-148597-162013, Fitness = 2.730977620730271
Rank 4: Genome = 2-1012109657-128034-1134-185-1791260826-139-140157, Fitness = 2.949334516415262
Rank 5: Genome = 10-1212065321084-1-1793409-126715-18974

## Storing Unique Individuals in a Set

In [10]:
print("\n=== Deduplication with Set ===")
ind2 = child1.copy_Individual(delete_fitness=False)  # Exact copy

# Mutate a third individual to likely get a different genome
ind3 = child1.copy_Individual(delete_fitness=False)
ind3.mutation()

ind_set = {child1, ind2, ind3}

for i, ind in enumerate(ind_set):
    print(f"Unique {i+1}: Genome = {ind.genome}, Fitness = {float(ind)}")

print(f"\nTotal unique individuals: {len(ind_set)}")


=== Deduplication with Set ===
Unique 1: Genome = 0-1110221704-19528-13689760453-1-11293574061-1-182, Fitness = 3.205545487495469

Total unique individuals: 1


# accessing the genome (removed)

In [11]:
print("Before genome swap:", child1.genome.course_assignments)
child1.genome.swap_course_assignments(0,0, 1)
print("After genome swap:", child1.genome.course_assignments)

Before genome swap: [ 1  7  0  4 -1  9  5  2  8 -1  3  6  8  9  7  6  0  4  5  3 -1 -1  1  2
  9  3  5  7  4  0  6  1 -1 -1  8  2]
After genome swap: [ 1  7  0  4 -1  9  5  2  8 -1  3  6  8  9  7  6  0  4  5  3 -1 -1  1  2
  9  3  5  7  4  0  6  1 -1 -1  8  2]


In [12]:
try:    
    child1.genome.swap_course_assignments(0,0, 10000)
except IndexError as e:
    print("Error:", e)

Error: Index out of range for course assignments.


# genetic algorithm

In [14]:
import random
from typing import List

import random
from typing import List

def tournament_selection(
    population: List[SolutionRD],
    maximization: bool = False,
    k: int = 3
) -> SolutionRD:
    """
    k-way tournament (with replacement), unified for minimization or maximization.

    Parameters
    ----------
    population : List[SolutionRD]
        Current population.
    maximization : bool, default=False
        If False, lower fitness is better (minimize). If True, higher fitness is better.
    k : int, default=3
        Tournament size (number of contestants).

    Returns
    -------
    SolutionRD
        The tournament winner.
    """
    # pick k random contestants (with replacement)
    contestants = random.choices(population, k=k)
    # choose best according to the mode
    if maximization:
        # higher fitness wins
        return max(contestants)
    else:
        # lower fitness wins
        return min(contestants)


def rank_selection(
    population: List[SolutionRD],
    maximization: bool = False
) -> SolutionRD:
    """
    Rank-based roulette selection.

    1. Sort population by fitness.
    2. Assign weights N, N-1, …, 1 (best gets highest weight).
    3. Sample one individual proportionally to its weight.

    Parameters
    ----------
    population : List[Individual]
        Current population.
    maximization : bool, default False
        If False, lower fitness is better; if True, higher fitness is better.

    Returns
    -------
    Individual
        The selected individual.
    """
    if maximization:
        return max(population)
    else:
        return min(population)
    
    

first_solution = SolutionRD(my_fitness, Genome, [logistic_mutation, social_mutation], [logistic_crossover_2, social_crossover])
population = [first_solution.copy_Individual() for _ in range(10)]

In [19]:
ranked_winner = rank_selection(population)
print("Ranked Winner:", ranked_winner.genome, "Fitness:", float(ranked_winner))

Ranked Winner: 01-1220184-1-110796253843167-19-15026214-1379-1085 Fitness: 3.4686852589641437


In [20]:
tournament_winner = tournament_selection(population)
print("Ranked Winner:", ranked_winner.genome, "Fitness:", float(ranked_winner))

Ranked Winner: 01-1220184-1-110796253843167-19-15026214-1379-1085 Fitness: 3.4686852589641437


In [ ]:
 
def genetic_algorithm(
    initial_population: List[SolutionRD],
    gen_count: int,
    selection_algorithm: list[callable],
    mutation_algorithms: list[callable],
    crossover_algorithms: list[callable],
    maximization: bool = False,
    xo_prob: float = 0.9,
    mut_prob: float = 0.2,
    social_mutation_prob: float = 0.2,
    social_crossover_prob: float = 0.2,
    elitism: bool = True,
    verbose: bool = False,
):
    first_solution = SolutionRD(my_fitness, Genome, mutation_algorithms, crossover_algorithms)
    first_solution.prob_social_mutation = social_mutation_prob
    first_solution.prob_social_crossover = social_crossover_prob